<a href="https://colab.research.google.com/github/az1fr3/Flowfield_Track/blob/master/StableAudioOpen_ByAzufr3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#
# _____ _        _     _         ___            _ _         _
#/  ___| |      | |   | |       / _ \          | (_)       | |          _
#\ `--.| |_ __ _| |__ | | ___  / /_\ \_   _  __| |_  ___   | |__  _   _(_)
# `--. \ __/ _` | '_ \| |/ _ \ |  _  | | | |/ _` | |/ _ \  | '_ \| | | |
#/\__/ / || (_| | |_) | |  __/ | | | | |_| | (_| | | (_) | | |_) | |_| |_
#\____/ \__\__,_|_.__/|_|\___| \_| |_/\__,_|\__,_|_|\___/  |_.__/ \__, (_)
#                                                                 __/ |
#                                                                |___/

#  █████╗ ███████╗ ██╗███████╗██████╗ ██████╗
# ██╔═██╗╚══███╔╝███║██╔════╝██╔══██╗╚════██╗
# ███████║  ███╔╝ ╚██║█████╗  ██████╔╝ █████╔╝
# ██╔══██║ ███╔╝   ██║██╔══╝  ██╔══██╗ ╚═══██╗
# ██║  ██║███████╗ ██║██║     ██║  ██║██████╔╝
# ╚═╝  ╚═╝╚══════╝ ╚═╝╚═╝     ╚═╝  ╚═╝╚═════╝
# ▒▒   ▓▒█░░▒▒ ▓░▒░▒ ▒ ░    ░ ▒▓ ░▒▓░
#   ▒   ▒▒ ░░░▒ ▒ ░ ▒ ░        ░▒ ░ ▒░
#   ░   ▒   ░ ░ ░ ░ ░ ░ ░      ░░   ░
#       ░  ░  ░ ░               ░
#           ░
#
# Sigueme para más! : https://linktr.ee/az1fr3
# Chequear el funcionamiento de NVIDIA, si te arroja algún error, en el menú ve a entorno de ejecución y activa la gpu Nvidia Tesla T4
# Para información sobre audigen visita su Github: https://github.com/facebookresearch/audiocraft
#Música opcional para esperar las descargas ^^ ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░#|
!wget -q "https://ia601209.us.archive.org/20/items/Chiptune_Songs_Archive/McFiredrill%20-%20Computer%20Trash/04%20-%20Vitamin%20D.mp3" -O keygenmusic.mp3
from IPython.display import Audio; display(Audio("keygenmusic.mp3", autoplay=True))
!nvidia-smi

#Instalar las dependencias
!pip install torch
!pip Install torchaudio
!pip install stable-audio-tools
!pip install einops

import torch
import torchaudio
from einops import rearrange
from stable_audio_tools import get_pretrained_model
from stable_audio_tools.inference.generation import generate_diffusion_cond
from google.colab import userdata
userdata.get('StableAudio')

#Este comando nos permite correr la app en la cpu sin necesidad de tener una GPU cuda.
device = "cuda" if torch.cuda.is_available() else "cpu"

# Descarga el modelo:
model, model_config = get_pretrained_model("audo/stable-audio-open-1.0") # Gracias a @Camenduru por el modelo de libre acceso!
sample_rate = model_config["sample_rate"]
sample_size = model_config["sample_size"]

model = model.to(device)


In [ ]:
# Setup General
conditioning = [{
    "prompt": "128 BPM Dub techno loop", #Escribe entre las comillas tu comando para generar
    "seconds_start": 0,
    "seconds_total": 47 # Modifica este numero para acceder
}]

# Generate stereo audio
output = generate_diffusion_cond(
    model,
    steps=100,
    cfg_scale=7,
    conditioning=conditioning,
    sample_size=sample_size,
    sigma_min=0.3,
    sigma_max=500,
    sampler_type="dpmpp-3m-sde",
    device=device
)

# Rearrange audio batch to a single sequence
output = rearrange(output, "b d n -> d (b n)")

# Peak normalize, clip, convert to int16, and save to file
output = output.to(torch.float32).div(torch.max(torch.abs(output))).clamp(-1, 1).mul(32767).to(torch.int16).cpu()
torchaudio.save("output.wav", output, sample_rate)

from IPython.display import Audio, display
display(Audio("/content/output.wav", autoplay=True))

#Si te gustó este notebook considera dejarme una propina en Kofi:
#o considera sumarte a mi Patreon y tener acceso a las herramientas mejoradas de AI: